## Luminosity comparison

In [33]:
from import_data import HMXB_parameters_Kaper
from import_data import HMXB_parameters
from import_data import supergiant_stellar_parameters
from import_data import falenga
from functions import scientific_notation
from functions import display_df
from import_data import stellar_params
from import_data import photometric_params

from astropy.constants import R_sun, L_sun, sigma_sb

import pandas as pd
import dataframe_image as dfi
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Markdown as md

pd.set_option('display.float_format', lambda x: '%.2E' % x)

In [34]:
# Import data
df_hmxb = HMXB_parameters()
df_sg_params = supergiant_stellar_parameters()
df_falenga = falenga()
df_stellar_params = stellar_params()
df_photometric_params = photometric_params()

df_L = pd.DataFrame({'id': df_hmxb['id'].copy(),
                              "L_true": [None] * len(df_hmxb['id']),
                              "L_expected": [None] * len(df_hmxb['id']),
                              "ST_short": [None] * len(df_hmxb['id']),
                              "ST": [None] * len(df_hmxb['id'])})

### Calculate the luminosity from photometric data
This is the luminosity we observe

In [35]:
for i in range(df_hmxb.shape[0]):
    # Spectral type of object
    spectral_type = df_hmxb.loc[i, "ST_short"]
    # Expected (B-V) of object based on spectral type
    BV0 = df_photometric_params[df_photometric_params['ST'] == spectral_type]["(B-V)0"].reset_index(drop=True).at[0]
    # Observed (B-V) of object based on simbad filters
    BVobs = df_hmxb.loc[i, '(B-V)obs']
    # Bolometric correction (BC)
    BCv = df_photometric_params[df_photometric_params['ST'] == spectral_type]["BCv"].reset_index(drop=True).at[0]

    # Calculate extinction
    Av = 3.2 * (BVobs - BV0)

    # Calculate Absulute magnitude (visual)
    Mv = df_hmxb.loc[i, 'mv'] - 5 * np.log10(df_hmxb.loc[i, 'distance'] * 1000) + 5 - Av

    # Calculate bolomatric absolute magnitude
    Mbol = Mv + BCv

    # Calculate the luminosity in solar luminosities
    L = 10**((Mbol - 4.74) / (-2.5))

    # Put L in luminosity dataframe
    df_L.loc[df_L["id"] == df_hmxb.loc[i, 'id'], "L_true"] = L
    # Put L in the dataframe
    df_hmxb.loc[i, 'luminosity'] = L


### Caclulate the luminosity from the radius and temperature of the optical star
This is the luminosity you would expect

In [36]:
for i in range(df_falenga.shape[0]):
    # Orbital separation
    a = df_falenga.loc[i, 'a'] * R_sun.value
    # Inclination
    inc = df_falenga.loc[i, 'i']
    # Semi eclipse angle
    theta_e = df_falenga.loc[i, 'semi_eclipse_angle']

    # Calculate Radius
    Rob = a * ((np.cos(np.deg2rad(inc)))**2 + (np.sin(np.deg2rad(inc)))**2 * (np.sin(np.deg2rad(theta_e)))**2)**(1/2)

    # Object id
    id = df_falenga.loc[i, 'id']
    # Spectral type
    spectral_type = df_hmxb[df_hmxb["id"] == str(id)]["ST_short"].reset_index(drop=True).at[0]
    # Spectral temperature
    Teff = df_stellar_params[df_stellar_params["ST"] == spectral_type]["Teff"].reset_index(drop=True).at[0]

    # Calculate Luminosity in solar luminosities
    L = (4 * np.pi * Rob**2 * sigma_sb.value * Teff**4) / L_sun.value

    # Put L in luminosity dataframe
    df_L.loc[df_L["id"] == id, "L_expected"] = L
    df_L.loc[df_L["id"] == id, "ST_short"] = spectral_type
    df_L.loc[df_L["id"] == id, "ST"] = df_hmxb.loc[df_hmxb["id"] == id, "ST"]

### Comparison of $L_{true}$ and $L_{expected}$(bolometric).
- The $L_{expected}$ is calculated from $L = 4 \pi R^{2} \sigma T_{eff}^{4}$, where:
    * $R$ is calculated from the semi-eclipse ange $\theta_{e}$
    * $T_{eff}$ is taken from the models of Martin et al. (2005a)
- The $L_{true}$ is determined from photometric data from Simbad and $(B-V)_{0}$ is from martins et al. (2006)

Calculate $\frac{L_{true}}{L_{expected}}$ (all luminosities are in solar units)

In [37]:
# Filter out all rows with none values
df_L_filtered = df_L.copy().dropna()
df_L_filtered.reset_index(drop=True)

# Calculate fraction
df_L_filtered["L_true/L_expected"] = df_L_filtered["L_true"] / df_L_filtered["L_expected"]

Result

In [38]:
df_L_filtered

,id,L_true,L_expected,ST_short,ST,L_true/L_expected
1,SMC X-1,5.53E+06,2.30E+05,O9I,O9.7Ia+,2.40E+01
2,LMC X-4,3.05E+06,7.62E+04,O8III,O8III,4.00E+01
3,Vela X-1,6.81E+05,7.88E+05,B0I,B0.5Ia,8.65E-01
4,Cen X-3,8.85E+06,1.36E+05,O9III,O9III,6.53E+01
6,4U1538-52,4.19E+05,8.70E+04,B0I,B0.2Ia,4.82E+00
7,4U1700-37,5.50E+05,8.89E+05,O6I,O6Iafcp,6.19E-01


### Comparison of $L_{true}$ and $L_{expected}$(model martins et. al (2005a)).

In [39]:
# Create new dataframe for the luminosities
df_L_model = pd.DataFrame({'id': df_hmxb['id'].copy(),
                              "L_true": df_L["L_true"].copy(),
                              "L_expected": [None] * len(df_hmxb['id']),
                              "ST_short": [None] * len(df_hmxb['id']),
                              "ST": [None] * len(df_hmxb['id'])})

Get the luminosity for the object from the models of Martin et. al (2005a)

In [40]:
for i in range(df_hmxb.shape[0]):
    # Spectral type
    spectral_type = df_hmxb.loc[i, 'ST_short']
    # Luminosity from model
    logL = df_stellar_params.loc[df_stellar_params["ST"] == spectral_type, "log(L/Lsun)"].reset_index(drop=True).at[0]
    L = 10**logL
    # Save luminosity in dataframe
    df_L_model.loc[i, "L_expected"] = L
    df_L_model.loc[i, "ST_short"] = spectral_type
    df_L_model.loc[i, "ST"] = df_hmxb.loc[i, "ST"]

In [41]:
# Calculate fraction
df_L_model["L_true/L_expected"] = df_L_model["L_true"] / df_L_model["L_expected"]

Result

In [42]:
df_L_model

,id,L_true,L_expected,ST_short,ST,L_true/L_expected
0,2S0114+650,2.06E+05,2.95E+05,B1I,B1Iae,6.99E-01
1,SMC X-1,5.53E+06,4.07E+05,O9I,O9.7Ia+,1.36E+01
2,LMC X-4,3.05E+06,2.24E+05,O8III,O8III,1.36E+01
3,Vela X-1,6.81E+05,3.47E+05,B0I,B0.5Ia,1.97E+00
4,Cen X-3,8.85E+06,1.62E+05,O9III,O9III,5.46E+01
5,GX301-2,6.58E+06,2.95E+05,B1I,B1.5Iaeq,2.23E+01
6,4U1538-52,4.19E+05,3.47E+05,B0I,B0.2Ia,1.21E+00
7,4U1700-37,5.50E+05,6.46E+05,O6I,O6Iafcp,8.53E-01
8,4U1907+09,1.32E+06,4.79E+05,O8I,O8.5Iab,2.76E+00
9,LMC X-1,1.53E+05,4.79E+05,O8I,O8(f)p,3.19E-01


### Comparison of $R_{true}$ and $R_{expected}$
where,
1. $R_{true}$ is calculated from the semi-eclipse angle.
2. $R_{expected}$ is based on the stellar parameters for the stars spectral type. Source Martin et. al (2005a). Get $L$ and $T_{eff}$ and calc R from $L = 4 \pi R^{2} \sigma T_{eff}^{4}$.

Both are calc. in terms of $R_{\odot}$

In [43]:
# Make radius dataframe
df_R = pd.DataFrame({'id': df_hmxb['id'].copy(),
                              "R_true": [None] * len(df_hmxb['id']),
                              "R_expected": [None] * len(df_hmxb['id']),
                              "ST_short": [None] * len(df_hmxb['id']),
                              "ST": [None] * len(df_hmxb['id'])})

1. $R_{true}$

In [45]:
for i in range(df_falenga.shape[0]):
    # Orbital separation
    a = df_falenga.loc[i, 'a'] * R_sun.value
    # Inclination
    inc = df_falenga.loc[i, 'i']
    # Semi eclipse angle
    theta_e = df_falenga.loc[i, 'semi_eclipse_angle']

    # Calculate Radius
    Rob = a * ((np.cos(np.deg2rad(inc)))**2 + (np.sin(np.deg2rad(inc)))**2 * (np.sin(np.deg2rad(theta_e)))**2)**(1/2)

    # Object id
    id = df_falenga.loc[i, 'id']
    # Spectral type
    spectral_type = df_hmxb[df_hmxb["id"] == str(id)]["ST_short"].reset_index(drop=True).at[0]

    # Put R in luminosity dataframe
    df_R.loc[df_R["id"] == id, "R_true"] = Rob / R_sun.value
    df_R.loc[df_R["id"] == id, "ST_short"] = spectral_type
    df_R.loc[df_R["id"] == id, "ST"] = df_hmxb.loc[df_hmxb["id"] == id, "ST"]

2. $R_{expected}$

In [ ]:
for i in range(df_hmxb.shape[0]):
    # Spectral type
    spectral_type = df_hmxb.loc[i, 'spectraltype(short)']

    # Luminosity from model
    logL = df_stellar_params.loc[df_stellar_params["ST"] == spectral_type, "log(L/Lsun)"].reset_index(drop=True).at[0]
    L = 10**logL

    # Effective temperature from model
    Teff = df_stellar_params.loc[df_stellar_params["ST"] == spectral_type, "Teff"].reset_index(drop=True).at[0]

    # Calculate the radius
    R = np.sqrt((L_sun.value / R_sun.value**2) * (L / (4 * np.pi * sigma_sb.value * Teff**4)))

    # Save luminosity in dataframe
    df_R.loc[i, "R_expected"] = R
    df_R.loc[i, "ST_short"] = spectral_type
    df_R.loc[i, "ST"] = df_hmxb.loc[i, "spectraltype"]

In [ ]:
# Calculate ratio
df_R["R_true/R_expected"] = df_R["R_true"] / df_R["R_expected"]

Result

In [ ]:
df_R_filtered = df_R.copy()
df_R_filtered = df_R_filtered.dropna().reset_index(drop=True)

df_R_filtered

,id,R_true,R_expected,ST_short,ST,R_true/R_expected
0,SMC X-1,1.63E+01,2.16E+01,O9I,O9.7Ia+,7.52E-01
1,LMC X-4,7.98E+00,1.37E+01,O8III,O8III,5.84E-01
2,Vela X-1,3.38E+01,2.25E+01,B0I,B0.5Ia,1.51E+00
3,Cen X-3,1.21E+01,1.32E+01,O9III,O9III,9.14E-01
4,4U1538-52,1.12E+01,2.25E+01,B0I,B0.2Ia,5.01E-01
5,4U1700-37,2.32E+01,1.98E+01,O6I,O6Iafcp,1.17E+00


In [ ]:
df_R.rename(columns={'R_true': r'$R_{true} (R_{\odot})$'})

,id,$R_{true} (R_{\odot})$,R_expected,ST_short,ST,R_true/R_expected
0,2S0114+650,None,2.20E+01,B1I,B1Iae,NaN
1,SMC X-1,1.63E+01,2.16E+01,O9I,O9.7Ia+,7.52E-01
2,LMC X-4,7.98E+00,1.37E+01,O8III,O8III,5.84E-01
3,Vela X-1,3.38E+01,2.25E+01,B0I,B0.5Ia,1.51E+00
4,Cen X-3,1.21E+01,1.32E+01,O9III,O9III,9.14E-01
5,GX301-2,None,2.20E+01,B1I,B1.5Iaeq,NaN
6,4U1538-52,1.12E+01,2.25E+01,B0I,B0.2Ia,5.01E-01
7,4U1700-37,2.32E+01,1.98E+01,O6I,O6Iafcp,1.17E+00
8,4U1907+09,None,2.09E+01,O8I,O8.5Iab,NaN
9,LMC X-1,None,2.09E+01,O8I,O8(f)p,NaN


In [46]:
display_df(df_hmxb)

| id         | Spec. Type   | Spec. Type (short)   |     ruwe |   $P[units]$ |   $P_{spin}[units]$ |   eclipseduration[units]$ |   $v_{rad}[km s^{-1}]$ |   $M_{ob}[M_{\odot}]$ |   $R_{ob}[R_{\odot}]$ |   $M_{x} [M_{\odot}]$ |   $p'' ['']$ |   $\delta p''['']$ |   $d[kpc]$ |   $d_{BJ}[kpc]$ |   $L_{ob}[L_{\odot}]$ |   $(B-V)_{obs}$ |     B |     V |   $(B-V)_{0}$ |   $BC_{v}$ |   $m_{v}$ |   $T_{eff}[K]$ |
|:-----------|:-------------|:---------------------|---------:|-------------:|--------------------:|--------------------------:|-----------------------:|----------------------:|----------------------:|----------------------:|-------------:|-------------------:|-----------:|----------------:|----------------------:|----------------:|------:|------:|--------------:|-----------:|----------:|---------------:|
| 2S0114+650 | B1Iae        | B1I                  | 0.944642 |    15.66     |     10008           |                       nan |                  -80   |                 16    |                   nan |                   nan |   0.196417   |          0.0112684 |    5.0912  |             nan |      206272           |            0.85 | 11.99 | 11.14 |         -0.28 |      -0.98 |     11.14 |          25119 |
| SMC X-1    | O9.7Ia+      | O9I                  | 1.00289  |   nan        |       nan           |                       nan |                  nan   |                nan    |                   nan |                   nan |   0.00399179 |          0.0247945 |  250.514   |             nan |           5.52668e+06 |           -0.15 | 13    | 13.15 |         -0.32 |      -3.03 |     13.15 |          32211 |
| LMC X-4    | O8III        | O8III                | 1.01448  |     1.4      |        13.5         |                       nan |                  nan   |                nan    |                   nan |                   nan |   0.0044352  |          0.0218083 |  225.469   |             nan |           3.04631e+06 |           -0.1  | 13.9  | 14    |         -0.32 |      -3.23 |     14    |          33963 |
| Vela X-1   | B0.5Ia       | B0I                  | 0.880197 |     0.000896 |       283           |                       nan |                   -3.2 |                 26    |                   nan |                   nan |   0.49624    |          0.015161  |    2.01515 |             nan |      681423           |            0.5  |  7.37 |  6.87 |         -0.31 |      -2.84 |      6.87 |          29512 |
| Cen X-3    | O9III        | O9III                | 1.0207   |     2.03     |         4.80188e-05 |                       nan |                   32   |                 20.2  |                   nan |                   nan |   0.138719   |          0.0136983 |    7.2088  |             nan |           8.84966e+06 |            2.13 | 14.4  | 12.27 |         -0.32 |      -3.03 |     12.27 |          32211 |
| GX301-2    | B1.5Iaeq     | B1I                  | 0.908531 |     0.0415   |       680           |                       nan |                    4.1 |                 43    |                   nan |                   nan |   0.250621   |          0.0159896 |    3.99009 |             nan |           6.57604e+06 |            2.04 | 12.7  | 10.66 |         -0.28 |      -0.98 |     10.66 |          25119 |
| 4U1538-52  | B0.2Ia       | B0I                  | 0.985791 |     3.73e-05 |       526.42        |                       nan |                 -158   |                 20    |                   nan |                   nan |   0.127959   |          0.0152523 |    7.81498 |             nan |      419426           |            1.8  | 16.3  | 14.5  |         -0.31 |      -2.84 |     14.5  |          29512 |
| 4U1700-37  | O6Iafcp      | O6I                  | 0.808192 |     3.41e-06 |       nan           |                       nan |                  -60   |                 46    |                   nan |                   nan |   0.632736   |          0.0259489 |    1.58044 |             nan |      550497           |            0.27 |  6.78 |  6.51 |         -0.33 |      -3.58 |      6.51 |          38282 |
| 4U1907+09  | O8.5Iab      | O8I                  | 0.883853 |     0.000838 |       437.5         |                       nan |                  nan   |                 27.8  |                   nan |                   nan |   0.232628   |          0.0731371 |    4.2987  |             nan |           1.32152e+06 |            3.06 | 19.41 | 16.35 |         -0.32 |      -3.23 |     16.35 |          33963 |
| LMC X-1    | O8(f)p       | O8I                  | 0.996263 |     3.90917  |       nan           |                       nan |                  nan   |                 31.79 |                   nan |                   nan |   0.0274709  |          0.0177021 |   36.4021  |             nan |      152705           |            0.3  | 14.8  | 14.5  |         -0.32 |      -3.23 |     14.5  |          33963 |
| Cyg X-1    | O9.7Iabpvar  | O9I                  | 0.967243 |     0.00056  |       nan           |                       nan |                   -7   |                 40.6  |                   nan |                   nan |   0.443902   |          0.0149494 |    2.25275 |             nan |      375860           |            0.81 |  9.72 |  8.91 |         -0.32 |      -3.03 |      8.91 |          32211 |

In [48]:
display_df(df_R_filtered)

| id        |   $R_{true}[R_{\odot}]$ |   $R_{expected}[R_{\odot}]$ | Spec. Type (short)   | Spec. Type   |   $\frac{R_{true}}{R_{expected}}$ |
|:----------|------------------------:|----------------------------:|:---------------------|:-------------|----------------------------------:|
| SMC X-1   |                16.2546  |                     21.6112 | O9I                  | O9.7Ia+      |                          0.752137 |
| LMC X-4   |                 7.97563 |                     13.6676 | O8III                | O8III        |                          0.583543 |
| Vela X-1  |                33.847   |                     22.4545 | B0I                  | B0.5Ia       |                          1.50736  |
| Cen X-3   |                12.094   |                     13.2295 | O9III                | O9III        |                          0.914169 |
| 4U1538-52 |                11.25    |                     22.4545 | B0I                  | B0.2Ia       |                          0.501012 |
| 4U1700-37 |                23.1986  |                     19.7667 | O6I                  | O6Iafcp      |                          1.17362  |